In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import math
import glob
from datetime import datetime

In [3]:
df = pd.read_csv("data/train_denorm_1M.zip")
df.head()

,CALL_TYPE,TAXI_ID,TIMESTAMP,tid,day_of_week,POLYLINE_LEN,lat,lon
0,C,20000380,1372636853,1372636853620000380,weekday,25,-8.610291,41.140746
1,C,20000380,1372636868,1372636853620000380,weekday,25,-8.610300,41.140755
2,C,20000380,1372636883,1372636853620000380,weekday,25,-8.610309,41.140890
3,C,20000380,1372636898,1372636853620000380,weekday,25,-8.613657,41.141358
4,C,20000380,1372636913,1372636853620000380,weekday,25,-8.614602,41.141484


In [4]:
len(df.tid.unique())

121312

In [5]:
#cambio i taxi id perchè il numero è troppo grande per l'implementazione java di Movelets

c = -1
new_tids = []

prec = -1

for row in tqdm(df.values):
    if row[3] != prec:
        prec = row[3]
        c +=1
    
    new_tids.append(c)
    #c+=1


df.tid = new_tids

df.head()

  0%|          | 0/5672929 [00:00<?, ?it/s]

,CALL_TYPE,TAXI_ID,TIMESTAMP,tid,day_of_week,POLYLINE_LEN,lat,lon
0,C,20000380,1372636853,0,weekday,25,-8.610291,41.140746
1,C,20000380,1372636868,0,weekday,25,-8.610300,41.140755
2,C,20000380,1372636883,0,weekday,25,-8.610309,41.140890
3,C,20000380,1372636898,0,weekday,25,-8.613657,41.141358
4,C,20000380,1372636913,0,weekday,25,-8.614602,41.141484


In [6]:
len(df.tid.unique())

121312

In [7]:
len(df)

5672929

In [8]:
 df = df[df.tid < df.tid.max()*.7]

In [9]:
len(df)

3989765

In [10]:
df0 = df[["tid", "CALL_TYPE", "lat", "lon", "TIMESTAMP"]]\
        .rename(columns={"lat": "c1", "lon": "c2", "TIMESTAMP": "t", "CALL_TYPE": "class"})

df0.head()

,tid,class,c1,c2,t
0,0,C,-8.610291,41.140746,1372636853
1,0,C,-8.610300,41.140755,1372636868
2,0,C,-8.610309,41.140890,1372636883
3,0,C,-8.613657,41.141358,1372636898
4,0,C,-8.614602,41.141484,1372636913


In [19]:
t_rocket = 0

start = datetime.now()

df0["pos"] = df0.groupby(['tid']).cumcount()

t_rocket = (datetime.now() - start).total_seconds() * 1000 #ms

In [20]:
start = datetime.now()

def padLastValue(df):
    array = df.values
    for i in tqdm(range(len(array))):
        for j in range(len(array[i])):
            if math.isnan(array[i][j]):
                array[i][j] =array[i][j-1]
    return pd.DataFrame(array)

percentile=1

print("LAT")
df_lat = pd.DataFrame()
df_lat = df0.groupby(['tid','pos'])['c1'].max().unstack()
df_lat = padLastValue(df_lat)

print("LON")
df_lon = pd.DataFrame()
df_lon = df0.groupby(['tid','pos'])['c2'].max().unstack()
df_lon = padLastValue(df_lon)

print("TIME")
df_time = pd.DataFrame()
df_time = df0.groupby(['tid','pos'])['t'].max().unstack()
df_time = padLastValue(df_time)

y = df0.groupby(['tid'])['class'].max()



from sktime.transformations.panel.rocket import Rocket
from sktime.datatypes._panel._convert import from_2d_array_to_nested

nested_lat = from_2d_array_to_nested(df_lat).rename(columns={0: "dim_0"})
nested_lon = from_2d_array_to_nested(df_lon).rename(columns={0: "dim_0"})
nested_time = from_2d_array_to_nested(df_time).rename(columns={0: "dim_0"})

nested = nested_lat
#nested = nested_UTMGridNorth

nested["dim_1"] = nested_lon["dim_0"]
nested["dim_2"] = nested_time["dim_0"]



t_rocket = (datetime.now() - start).total_seconds() * 1000 #ms

LAT


  0%|          | 0/84918 [00:00<?, ?it/s]

LON


  0%|          | 0/84918 [00:00<?, ?it/s]

TIME


  0%|          | 0/84918 [00:00<?, ?it/s]

In [21]:
df0.head()

,tid,class,c1,c2,t,latLon,pos
0,0,C,-8.610291,41.140746,1372636853,-8.610291 41.140746,0
1,0,C,-8.610300,41.140755,1372636868,-8.6103 41.140755,1
2,0,C,-8.610309,41.140890,1372636883,-8.610309 41.14089,2
3,0,C,-8.613657,41.141358,1372636898,-8.613657 41.141358,3
4,0,C,-8.614602,41.141484,1372636913,-8.614602 41.141484,4


In [22]:
start = datetime.now()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(nested, y, test_size=.3, stratify=y)

rocket = Rocket(n_jobs=-1, num_kernels=10000)  # by default, ROCKET uses 10,000 kernels
rocket.fit(X_train)

X_train_transform = rocket.transform(X_train)
X_test_transform = rocket.transform(X_test)

t_rocket = (datetime.now() - start).total_seconds() * 1000 #ms

In [23]:
t_rocket

798781.589

In [25]:
t_rocket

798781.589

In [26]:
%%time

rf = clf.best_estimator_
#clf = tree.DecisionTreeClassifier(max_depth=2)

rf.fit(X_train_transform, y_train)

from sklearn import metrics
y_pred=rf.predict(X_test_transform)


from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           A       0.56      0.19      0.28      5579
           B       0.56      0.94      0.70     12969
           C       0.63      0.18      0.27      6928

    accuracy                           0.57     25476
   macro avg       0.58      0.43      0.42     25476
weighted avg       0.58      0.57      0.49     25476

CPU times: user 2h 55min 35s, sys: 1.06 s, total: 2h 55min 36s
Wall time: 14min 43s


In [27]:
## Movelets

In [28]:
import random
import os


def prepareForShapelet(df=pd.DataFrame, trajectoryIDAttribute= None, targetAttribute=None,
                       timestampAttribute=None, latAttr=None, lonAttr=None, trainPerc=.80, trajPerc=1.0, maxPoints=None):
    attributes = [timestampAttribute, "latLon"]+ [x for x in list(df.columns) if x not in [trajectoryIDAttribute, targetAttribute, timestampAttribute, latAttr, lonAttr]] #mi assicuro di avere il timestam come primo elemento
    if(os.path.exists("train") | os.path.exists("test")):
        print("Le cartelle train e test esistono già!")
        return

    df["latLon"] = df[latAttr].astype(str) + " " + df[lonAttr].astype(str)

    ids = list(df[trajectoryIDAttribute].unique())
    #ids = random.sample(ids, round(len(ids)*trajPerc))
    ids = ids[:round(len(ids)*trajPerc)]
    ids_train = random.sample(ids, round(trainPerc*len(ids)))
    ids_test = [x for x in ids if x not in ids_train]

    os.makedirs("train")
    os.makedirs("test")

    for id in tqdm(ids_train):
        df_id = df[df[trajectoryIDAttribute] == id]
        classe = df_id[targetAttribute].iloc[0]
        if maxPoints is not None:
            df[df[trajectoryIDAttribute] == id][attributes].head(maxPoints).to_csv(F"train/{id} s{id} c{classe}.r2", index=False, header=False)
        else:
            df[df[trajectoryIDAttribute] == id][attributes].to_csv(F"train/{id} s{id} c{classe}.r2", index=False, header=False)

    for id in tqdm(ids_test):
        df_id = df[df[trajectoryIDAttribute] == id]
        if maxPoints is not None:
            df_id = df_id.head(maxPoints)
        classe = df_id[targetAttribute].iloc[0]
        if maxPoints is not None:
            df[df[trajectoryIDAttribute] == id][attributes].head(maxPoints).to_csv(F"test/{id} s{id} c{classe}.r2", index=False, header=False)
        else:
            df[df[trajectoryIDAttribute] == id][attributes].to_csv(F"test/{id} s{id} c{classe}.r2", index=False, header=False)

prepareForShapelet(df0, "tid", "class", "t", "c1", "c2", trainPerc=.7, trajPerc=1)

Le cartelle train e test esistono già!


In [12]:
#heap space

In [11]:
## Geolet

In [12]:
import geolib.geohash
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import pandas as pd
from tqdm.auto import tqdm
from cri98tj.partitioners.Geohash_partitioner import Geohash_partitioner
from cri98tj.normalizers.FirstPoint_normalizer import FirstPoint_normalizer
from cri98tj.selectors.RandomInformationGain_selector import RandomInformationGain_selector
from cri98tj.distancers.Euclidean_distancer import Euclidean_distancer
from sklearn.model_selection import train_test_split
from cri98tj.distancers.Euclidean_distancer import euclideanBestFitting

spatioTemporalCols = ["c1", "c2", "t"]

tid_train, tid_test, _, _ = train_test_split(df0.groupby(by=["tid"]).max().reset_index()["tid"],
                                                        df0.groupby(by=["tid"]).max().reset_index()["class"],
                                                        test_size=.3,
                                                        stratify=df0.groupby(by=["tid"]).max().reset_index()["class"],
                                                        random_state=3)

In [19]:
%%time

from datetime import datetime
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from cri98tj.distancers.InterpolatedRootDistance_distancer import InterpolatedRootDistance_distancer, \
    InterpolatedRootDistanceBestFitting

precision = 5
n_jobs = 24
verbose = True



df1 = df0[["tid", "class", "c1", "c2", "t"]].copy()

normalizer = FirstPoint_normalizer(spatioTemporalColumns=spatioTemporalCols, fillna=None, verbose=verbose)
distancer = InterpolatedRootDistance_distancer(normalizer=normalizer, spatioTemporalColumns=spatioTemporalCols, n_jobs=n_jobs, verbose=verbose)
partitioner = Geohash_partitioner(precision=precision, spatioTemporalColumns=spatioTemporalCols, verbose=verbose)

start = datetime.now()
part = partitioner.fit_transform(df1[df1.tid.isin(tid_train)].values)
selector = RandomInformationGain_selector(top_k=50, bestFittingMeasure=InterpolatedRootDistanceBestFitting, 
                                          movelets_per_class=3000, trajectories_for_orderline=500, 
                                          n_jobs=n_jobs, spatioTemporalColumns=spatioTemporalCols, 
                                          normalizer=normalizer, verbose=verbose)
shapelets = selector.fit_transform(part)
_, dist_np = distancer.fit_transform((df1.values, shapelets))

Encoding 2793946 points with precision 5


  0%|          | 0/2793946 [00:00<?, ?it/s]

Cutting sub-trajectories length at 42.0 over 115
Pivoting tables


  0%|          | 0/53608 [00:00<?, ?it/s]

Cutting sub-trajectories length at 70.0 over 118
Pivoting tables


...

0.	 score=0.09495440516786613
1.	 score=0.09353453482209306
2.	 score=0.09283467381287758
3.	 score=0.09201182673771235
4.	 score=0.09133926527022496
5.	 score=0.0902442019900711
6.	 score=0.08984293469130233
7.	 score=0.08908368503559894
8.	 score=0.08756923476878398
9.	 score=0.08716060297992256
10.	 score=0.0868032026200658
11.	 score=0.08582418604675013
12.	 score=0.08410600024727022
13.	 score=0.0840100499706602
14.	 score=0.0828501060882707
15.	 score=0.08224043016241622
16.	 score=0.08166264245434762
17.	 score=0.0798849790058318
18.	 score=0.07894100239603152
19.	 score=0.07865518071900901
20.	 score=0.07865302009522313
21.	 score=0.07855709287464396
22.	 score=0.07827336395940065
23.	 score=0.07809621572745651
24.	 score=0.07782222459094479
25.	 score=0.07739861402802029
26.	 score=0.0769897391755312
27.	 score=0.07693705375398041
28.	 score=0.07674933582584753
29.	 score=0.07672691771433127
30.	 score=0.07525206742904533
31.	 score=0.07520908200268339
32.	 score=0.07453779011

  0%|          | 0/3989765 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

CPU times: user 2min 6s, sys: 10.7 s, total: 2min 16s
Wall time: 44min 27s


In [21]:
dist_np_df = pd.DataFrame(dist_np)
X = dist_np_df.drop(columns=[0]).values
y = dist_np_df[0].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y, random_state=3)

In [22]:
rf = RandomForestClassifier(criterion='entropy', max_depth=22, n_estimators=1500, n_jobs=-1)#clf.best_estimator_


rf.fit(X_train, y_train)

from sklearn.metrics import classification_report

y_pred = rf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           A       0.58      0.15      0.24      5579
           B       0.56      0.94      0.70     12969
           C       0.60      0.21      0.31      6928

    accuracy                           0.57     25476
   macro avg       0.58      0.43      0.42     25476
weighted avg       0.58      0.57      0.50     25476

